In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv
import lxml

# Regex
regex_url = r"(http(s?):\/\/(www)?[A-Za-z0-9-?\/\.]+)"
regex_hastags = r"(#[A-Za-z0-9]{2,})"

# File url
file_url = '../datasets/everydayhealth.txt'

# Basic list
csv_header = ['tweetid','date','title']
frame_header = ['tweetid','date','title','what']

# Support scripts
%run -i scripts.py

In [2]:
frame = pd.read_csv(file_url, sep='|', header= None, names = csv_header)
frame.head()

,tweetid,date,title
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a..."
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,What a heart attack feels like in women (it's ...
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...


In [3]:
frame.shape

(3238, 3)

In [4]:
frame.loc[:,'title'][4]

"#McDonalds oatmeal has almost 7 teaspoons of sugar in it! More healthy fast food that isn't: http://bit.ly/GGM5bA"

In [5]:
# Split Url & Text
frame['url'] = frame['title'].map(lambda row: get_url(regex_url, row))
frame['title'] = frame['title'].map(lambda row: split_txt_form_url(regex_url, row))
frame.head()

http://bit.ly/VymaIc
http://bit.ly/TeQPqk
http://trib.al/LDbxW6
http://bit.ly/InrWrC
http://bit.ly/GGM5bA
http://trib.al/4eRjME
http://bit.ly/yWptJ7
http://bit.ly/R7OpqA
http://bit.ly/HxUJEn
http://bit.ly/HXuSpU
http://bit.ly/yTBw80
http://bit.ly/xbkHC0
http://trib.al/wMjJUG
http://bit.ly/YLPgnc
http://trib.al/iLlHOR
http://bit.ly/wyXSKB
http://bit.ly/HIMacH
http://bit.ly/Hb4f20
http://trib.al/YXvpJx
http://bit.ly/U4HEdQ
http://trib.al/NxtEcR
http://bit.ly/T2lpU6
http://bit.ly/YnEeSn
http://trib.al/Z0JjTe
http://bit.ly/TngoWp
http://bit.ly/zutyZW
http://bit.ly/UlfGut
http://bit.ly/yCNop5
http://paper.li/EverydayHealth/1340975022
http://bit.ly/155TOrX
http://bit.ly/Vnwmkm
http://bit.ly/xrJsuW
http://ow.ly/hTS7J
http://bit.ly/11t6Hy7
http://bit.ly/NNsFUd
http://bit.ly/10dj5kk
http://bit.ly/VIs4TP
http://ow.ly/hTS2e
http://trib.al/eD95Gw
http://bit.ly/15w0Uam
http://bit.ly/155GPGE
http://bit.ly/ZUrqV8
http://bit.ly/UKV7Yh
http://bit.ly/VQCBj8
http://bit.ly/xjp08G
http://ow.ly/hTzaV
http:/

http://trib.al/4eRjME
http://bit.ly/yWptJ7
http://bit.ly/R7OpqA
http://bit.ly/HxUJEn
http://bit.ly/HXuSpU
http://bit.ly/yTBw80
http://bit.ly/xbkHC0
http://trib.al/wMjJUG
http://bit.ly/YLPgnc
http://trib.al/iLlHOR
http://bit.ly/wyXSKB
http://bit.ly/HIMacH
http://bit.ly/Hb4f20
http://trib.al/YXvpJx
http://trib.al/TKJnnX
http://trib.al/NxtEcR
http://bit.ly/InrVDY
http://bit.ly/T2lpU6
http://trib.al/Z0JjTe
http://bit.ly/TngoWp
http://bit.ly/zutyZW
http://bit.ly/Hqi6U0
http://bit.ly/yCNop5
http://paper.li/EverydayHealth/1340975022
http://trib.al/0RhvPe
http://bit.ly/xrJsuW
http://bit.ly/NNsFUd
http://bit.ly/TeQOmo
http://bit.ly/SBBOBz
http://trib.al/eD95Gw
http://bit.ly/GHpa9c
http://bit.ly/HtE4n7
http://trib.al/FLc630
http://bit.ly/xjp08G
http://bit.ly/R7OQBt
http://bit.ly/ImcQm9
http://trib.al/oUQLCA
http://trib.al/5WEXbi
http://trib.al/JpXYFR
http://trib.al/GoigeC
http://bit.ly/xEcHYf
http://bit.ly/FQ0Gwp
http://bit.ly/xor8bT
http://trib.al/sfdgoL
http://trib.al/u2uaHq
http://trib.al/mJs

http://bit.ly/xRMj4d
http://bit.ly/YaWIHg
http://bit.ly/YaWD6D
http://bit.ly/Y7ppmP
http://bit.ly/Z8tbTq
http://bit.ly/T8sZwC
http://bit.ly/yxxa6j
http://trib.al/C1hOur
http://bit.ly/Ve8ylk
http://bit.ly/TngiOy
http://bit.ly/HIM4C0
http://bit.ly/12JBB4y
http://bit.ly/12JzRZj
http://bit.ly/zjC5kE
http://bit.ly/14ETdNq
http://bit.ly/yGFG9X
http://bit.ly/yD3UJT
http://bit.ly/12JpeFD
http://bit.ly/SHHWIE
http://www.everydayhealth.com/healthtalk/healthtalk-with-julieanna-hever-and-dreena-burton-how-to-go-vegan-5813.aspx
http://bit.ly/Z7YKwG
http://bit.ly/UHJoK4
http://bit.ly/Q36M49
http://bit.ly/JpSruK
http://trib.al/g9Kaxs
http://bit.ly/xAGADb
http://fb.me/1YKAJ713n
http://bit.ly/ArazCT
http://bit.ly/WYdNVK
http://bit.ly/Y6DvVo
http://bit.ly/WYddHx
http://trib.al/gDj1EL
http://bit.ly/V3qstw
http://bit.ly/xrlcjQ
http://bit.ly/R7Pi2u
http://bit.ly/TngpJS
http://bit.ly/y5CzCb
http://bit.ly/WlcNIn
http://bit.ly/GH7MBt
http://bit.ly/W4SaEx
http://bit.ly/A7BLqe
http://bit.ly/HIM5FX
http://bit.ly

http://bit.ly/Vs1XTS
http://bit.ly/13IYao7
http://bit.ly/U4nulA
http://bit.ly/10hUwTj
http://bit.ly/Hb4c6a
http://www.cspinet.org/new/201301161.html
http://bit.ly/TkpNOw
http://bit.ly/Q36Kth
http://bit.ly/UT3MX7
http://cbsn.ws/XdU2Gz
http://bit.ly/10C438E
http://trib.al/bxHEXV
http://bit.ly/1027mQF
http://bit.ly/XdEceY
http://bit.ly/A3zxlB
http://bit.ly/yoJh7c
http://bit.ly/ytLAdX
http://bit.ly/GGFvSs
http://bit.ly/10h3lNc
http://bit.ly/U3WJxE
http://bit.ly/U3WItv
http://ow.ly/gTL1B
http://trib.al/pngiYS
http://bit.ly/GXXMbQ
http://ow.ly/gQqcD
http://bit.ly/yUjPNo
http://ow.ly/gQqcD
http://bit.ly/RUdSq2
http://bit.ly/JpSxCG
http://bit.ly/Uwrdr0
http://bit.ly/USFuwC
http://bit.ly/UwpIci
http://bit.ly/Vqlynw
http://bit.ly/wfZaXV
http://bit.ly/FPRzvR
http://bit.ly/GH5RNm
http://trib.al/3XqqeM
http://trib.al/E0O0bt
http://bit.ly/H8gbnd
http://bit.ly/yKGhJq
http://trib.al/jvBuHC
http://bit.ly/GHuKZ8
http://bit.ly/GEA0Pj
http://bit.ly/InrY2M
http://trib.al/hu9cyY
http://bit.ly/InFrW8
http://

,tweetid,date,title,url
0,304596701757464576,Thu Feb 21 14:21:27 +0000 2013,#FastFood Makes Up 11 Percent of #Calories in ...,http://bit.ly/VymaIc
1,304595191329853441,Thu Feb 21 14:15:27 +0000 2013,"10 snacks to help you lose weight, burn fat, a...",http://bit.ly/TeQPqk
2,304587659018371072,Thu Feb 21 13:45:31 +0000 2013,10 foods that boost your skin AND slim your wa...,http://trib.al/LDbxW6
3,304580073380524032,Thu Feb 21 13:15:22 +0000 2013,at a heart attack feels like in women (it's di...,http://bit.ly/InrWrC
4,304572560270573569,Thu Feb 21 12:45:31 +0000 2013,#McDonalds oatmeal has almost 7 teaspoons of s...,http://bit.ly/GGM5bA


In [6]:
frame.loc[:,'url'][0]

'http://bit.ly/VymaIc'

In [7]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3238 entries, 0 to 3237
Data columns (total 4 columns):
tweetid    3238 non-null int64
date       3238 non-null object
title      3238 non-null object
url        3238 non-null object
dtypes: int64(1), object(3)
memory usage: 101.3+ KB


In [8]:
#Author:  //a[@class="article_link"]/span/text()
#Content: //div[@class="article-body"]/p/text()

In [9]:
frame_urls = frame['url'].values

In [10]:
# Run for this jupyter notebook
title_attempt = True
for key,url in enumerate(frame_urls):

    #     url = 'http://bit.ly/VymaIc'
    try:
        # Start the connection
        print(url)
        if len(url):
            if "http://" not in url:
                url = "http://" + url
            res = requests.get(url)

            # Check status header
            if res.status_code == 200:
                print("Header: ", 200)

                # Initialize BeautifulSoup()
                soup = BeautifulSoup(res.content, 'lxml')

                # Get return content as text
                # text_html = res.text

                # Add columns to dataframe
                if title_attempt:
                    df_temp = pd.DataFrame(columns=['author','content','tags'])
                    frame = pd.concat([frame,df_temp])
                    title_attempt = False

                ahref = soup.find('a',{'class':'article_link'})
                if ahref:
                    author = ahref.text            
        #             frame['author'][key] = author[:author.index(",")].lstrip()
                    frame['author'][key] = author.lstrip()
                else:
                    frame['author'] = ''
                frame['content'][key] = soup.find('div',{'class':'article-body'})
            else:
                print("[ERROR]: ", res.status_code)
        else:
            frame['author'][key] = ''
            frame['content'] = ''
        frame['tags'][key] = get_hastags_out(regex_hastags,frame['title'][key])
        frame['title'][key] = stript_hastags(frame['title'][key])
    except (RuntimeError, TypeError, NameError):
        pass

http://bit.ly/VymaIc
Header:  200


scripts.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  # Get hastags out of string.
scripts.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
scripts.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return sample.replace("#",'')
scripts.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#i

http://bit.ly/TeQPqk
Header:  200
http://trib.al/LDbxW6
Header:  200
http://bit.ly/InrWrC
Header:  200
http://bit.ly/GGM5bA
Header:  200
http://trib.al/4eRjME
Header:  200
http://bit.ly/yWptJ7
Header:  200
http://bit.ly/R7OpqA
Header:  200
http://bit.ly/HxUJEn
Header:  200
http://bit.ly/HXuSpU
Header:  200
http://bit.ly/yTBw80
Header:  200
http://bit.ly/xbkHC0
Header:  200
http://trib.al/wMjJUG
Header:  200
http://bit.ly/YLPgnc
Header:  200
http://trib.al/iLlHOR
Header:  200
http://bit.ly/wyXSKB
Header:  200
http://bit.ly/HIMacH
Header:  200
http://bit.ly/Hb4f20
Header:  200
http://trib.al/YXvpJx
Header:  200
http://bit.ly/U4HEdQ
Header:  200
http://trib.al/NxtEcR
Header:  200
http://bit.ly/T2lpU6
Header:  200
http://bit.ly/YnEeSn
Header:  200
http://trib.al/Z0JjTe
Header:  200
http://bit.ly/TngoWp
Header:  200
http://bit.ly/zutyZW
Header:  200
http://bit.ly/UlfGut
Header:  200
http://bit.ly/yCNop5
Header:  200
http://paper.li/EverydayHealth/1340975022
Header:  200
http://bit.ly/155TOr

scripts.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Scrapt content base on xpath command.


http://www.montefiore.org/heartmonth
Header:  200





http://www.montefiore.org/heartmonth
Header:  200
http://twitter.com/TheHeartTruth/status/304303368908713984/photo/1
Header:  200


http://bit.ly/YGA5JV
Header:  200
http://owl.li/hTrms
Header:  200



http://owl.li/hTqSb
Header:  200




http://trib.al/oUQLCA
Header:  200










http://bit.ly/XlaNnI
Header:  200























http://trib.al/5WEXbi
Header:  200




















http://bit.ly/12NfIlo
Header:  200
http://trib.al/JpXYFR
Header:  200

http://bit.ly/15vlx6l
Header:  200

http://bit.ly/11cqFMg
Header:  200

http://bit.ly/xEcHYf
Header:  200
http://bit.ly/FQ0Gwp
Header:  200
http://ow.ly/hSXDe
Header:  200

http://bit.ly/xor8bT
Header:  200
http://trib.al/sfdgoL
Header:  200

http://trib.al/u2uaHq
Header:  200
http://ow.ly/hRjQ0
Header:  200
http://trib.al/mJs4zK
Header:  200
http://bit.ly/T8t17J
Header:  200
http://bit.ly/AuMaGe
Header:  200
http://bit.ly/Ht7o0X
Header:  200
http://ow.ly/hRjQ0
Header:  200

Header:  200
http://bit.ly/zpPHB9
Header:  200
http://bit.ly/HjtW3p
Header:  200
http://bit.ly/Sv10Vx
Header:  200
http://trib.al/VyZuh4
Header:  200
http://bit.ly/ULP4Qd
Header:  200
http://trib.al/yjuERY
Header:  200
http://bit.ly/H6vvyd
Header:  200
http://bit.ly/HTuc7v
Header:  200
http://trib.al/0fmbnj
[ERROR]:  404
http://bit.ly/NNrsfC
Header:  200
http://bit.ly/PUeNIN
Header:  200
http://trib.al/4eZlmR
Header:  200
http://bit.ly/Hb48Du
Header:  200
http://trib.al/UGs0yq
Header:  200
http://bit.ly/13Eaw0J
Header:  200
http://bit.ly/HWQm6s
Header:  200
http://bit.ly/SHI0Ij
Header:  200
http://trib.al/1mW7UU
Header:  200
http://bit.ly/I5zICy
Header:  200
http://bit.ly/XLI0bM
Header:  200
http://bit.ly/Ri4QTd
Header:  200
http://trib.al/NEdwsy
Header:  200
http://bit.ly/T2lnvm
Header:  200
http://bit.ly/SBBMd5
Header:  200
http://bit.ly/ITTuAI
Header:  200
http://paper.li/EverydayHealth/1340975022
Header:  200
http://bit.ly/O2DQrl
Header:  200
http://bit.ly/11Lsi54
Header:  200
http

ConnectionError: HTTPConnectionPool(host='qa1.everydayhealth.com', port=80): Max retries exceeded with url: /emotional-health/0614/what-do-your-shoes-say-about-your-personality.aspx?xid=tw_everydayhealth_sf (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000002AE49D44C88>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed',))

In [ ]:
frame.head()

In [ ]:
frame.shape

In [ ]:
# author = author[:author.index(",")].lstrip()
# ide = author.index(",")
# print(ide)
# print(author)
# frame['url'][0]

In [ ]:
# frame.iloc[:,2][0]

In [ ]:
# hastags = get_hastags_out(regex_hastags, frame.iloc[:,2][0])
# print(hastags)
# text = stript_hastags(frame.iloc[:,2][0])
# print(text)

In [ ]:
frame['Content'] = frame['Content'].apply(lambda row: )

In [12]:
frame.to_csv('./everyday_scrapt.csv', sep='|', encoding='utf-8')